In [17]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
from torchvision import models
from torchvision.models import DenseNet121_Weights

In [18]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5), # Flip image horizontally with 50% probability
    transforms.RandomRotation(degrees=30),  # Rotate image randomly up to ±30 degrees
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)), # Random crop and resize
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Adjust brightness, contrast
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])
])

In [19]:
train_dataset = datasets.ImageFolder(root="train", transform=transform)
test_dataset = datasets.ImageFolder(root="test", transform=transform)

In [20]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 2000
    Root location: train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.1, 0.1))
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [21]:
train_dataset.classes

['cats', 'dogs']

In [22]:
# dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [23]:
model = models.densenet121(weights=DenseNet121_Weights.DEFAULT)

In [24]:
num_classes = len(train_dataset.classes)  # Get number of classes

# Modify last FC layer for custom classification
model.fc = nn.Linear(2048, num_classes)

In [25]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [27]:
torch.cuda.memory_allocated()  # Checks allocated memory

164563968

In [28]:
torch.cuda.empty_cache()

In [29]:
model = model.to(device)

In [30]:
# Define Loss & Optimizer
loss_fc = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [31]:
# training loop

import time
start_time = time.time()

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for features , label in train_loader:
        features, label = features.to(device), label.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = loss_fc(outputs, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

end_time = time.time()
total_time = end_time - start_time
print(f"Total training time is {total_time / 60} Minutes!")

Epoch [1/10], Loss: 0.8729
Epoch [2/10], Loss: 0.2020
Epoch [3/10], Loss: 0.1388
Epoch [4/10], Loss: 0.1319
Epoch [5/10], Loss: 0.1172
Epoch [6/10], Loss: 0.1404
Epoch [7/10], Loss: 0.1168
Epoch [8/10], Loss: 0.1054
Epoch [9/10], Loss: 0.0794
Epoch [10/10], Loss: 0.0866
Total training time is 48.42381890217463 Minutes!


In [32]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for feature, labels in test_loader:
        feature, labels = feature.to(device), labels.to(device)
        outputs = model(feature)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

Validation Accuracy: 92.00%
